In [1]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


定义网络

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用模型，我们将输入数据传递给它。这将执行模型的forward，以及一些后台操作。不要直接调用model.forward() !

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


模型层

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten  
把2D的28x28图片转化为一个784个像素值的连续数组

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear  
使用其存储的权重和偏置值对输入进行一个线性变换

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU  
非线性激活函数在模型的输入和输出之间创建复杂的映射，它们应用于线性变换后引入非线性，帮助神经网络学习各种各样的现象。

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.6175e-01,  2.2003e-01, -2.8839e-01, -9.8895e-05, -3.2185e-02,
          2.7835e-01,  3.5125e-01, -2.6175e-01, -2.6731e-01,  7.1005e-02,
          3.0541e-01, -8.8769e-01,  9.9917e-02, -2.1116e-01,  3.0926e-01,
         -1.7012e-01, -8.6330e-02, -6.9018e-01, -6.5629e-01, -3.9315e-01],
        [-4.0696e-01,  6.6927e-02, -3.1480e-01, -1.2830e-02, -3.0320e-01,
         -1.8560e-01,  3.7451e-02, -2.5285e-01, -1.5362e-01,  2.7031e-01,
          4.3419e-01, -6.5314e-01,  9.9530e-02, -6.7920e-02,  1.3081e-01,
         -2.0238e-01, -5.1028e-02, -5.7171e-01, -1.3553e-01,  2.1053e-01],
        [-1.8415e-01, -6.7414e-02, -2.0869e-01,  1.5349e-01, -3.2052e-03,
          3.8982e-02,  2.0596e-01, -3.4277e-01,  1.7266e-01,  6.9829e-02,
          2.9444e-01, -6.9156e-01,  2.7304e-01, -1.6372e-01,  3.6503e-01,
         -3.8827e-01, -1.7436e-01, -7.4231e-01, -2.4793e-01, -1.1239e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2200, 0.0000, 0.0000, 0.0000,

#### nn.Sequential  
nn.Sequential是一个包含多个模块的有序容器。数据按照定义的顺序通过所有模块。  

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

#### nn.Softmax  
神经网络最后一个线性层返回的原始logits值范围为负无穷到正无穷，softmax将他们压缩到[0,1]范围内，表示模型对每个类的预测概率，dim参数指明沿着该维度数值和为1.  

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1108, 0.0733, 0.0697, 0.1166, 0.1079, 0.1052, 0.1151, 0.1317, 0.0796,
         0.0900],
        [0.1056, 0.0785, 0.0730, 0.1090, 0.1107, 0.1136, 0.1153, 0.1263, 0.0814,
         0.0867],
        [0.1033, 0.0826, 0.0675, 0.1195, 0.1141, 0.1085, 0.1090, 0.1287, 0.0854,
         0.0813]], grad_fn=<SoftmaxBackward0>)


模型参数

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0197, -0.0213, -0.0010,  ..., -0.0299,  0.0345,  0.0148],
        [-0.0257,  0.0061,  0.0345,  ...,  0.0114, -0.0109,  0.0247]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0315, 0.0021], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0045, -0.0386, -0.0414,  ..., -0.0225, -0.0259, -0.0352],
        [-0.0144, -0.0240, -0.0143,  ..., -0.0370,  0.0009, -0.0321]],
       device='cuda:0', grad_fn=<Slic